<a href="https://colab.research.google.com/github/multitrack-collector/gpt-4chan_notebook/blob/main/gpt_4chan_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

See if you have a *Tesla T4* GPU by running command below. If you don't have one, you have to **change your runtime** under the runtime tab above.

In [ ]:
!nvidia-smi

Thu Jul 25 02:36:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Inital Setup

In [ ]:
!pip install -U torrentp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 86.6 MB/s eta 0:00:00


# float16 (gpu)

In [ ]:
!wget https://archive.org/download/gpt4chan_model_float16/gpt4chan_model_float16_archive.torrent
!wget https://github.com/Aspie96/gpt-4chan-model/raw/float16/config.json



--2024-07-25 02:36:35--  https://archive.org/download/gpt4chan_model_float16/gpt4chan_model_float16_archive.torrent
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia902203.us.archive.org/16/items/gpt4chan_model_float16/gpt4chan_model_float16_archive.torrent [following]
--2024-07-25 02:36:35--  https://ia902203.us.archive.org/16/items/gpt4chan_model_float16/gpt4chan_model_float16_archive.torrent
Resolving ia902203.us.archive.org (ia902203.us.archive.org)... 207.241.228.63
Connecting to ia902203.us.archive.org (ia902203.us.archive.org)|207.241.228.63|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59655 (58K) [application/x-bittorrent]
Saving to: ‘gpt4chan_model_float16_archive.torrent’

gpt4chan_model_floa 100%[===================>]  58.26K  --.-KB/s    in 0.02s   

2024-07-25 02:36:35 (3.34 MB/s) - ‘gpt4chan_model_f

In [ ]:
from torrentp import TorrentDownloader
torrent_file = TorrentDownloader("gpt4chan_model_float16_archive.torrent", '.')
await torrent_file.start_download()
!cp config.json ./gpt4chan_model_float16/


Size: 12106074.81 MB
Saving as: gpt4chan_model_float16
 6022.4 Kb/s | up: 223.2 Kb/s | status: seeding | peers: 0  |####################| 100%
Downloaded successfully.


In [ ]:
from transformers import GPTJForCausalLM, AutoTokenizer
import torch

from transformers import GPTJForCausalLM
import torch

model = GPTJForCausalLM.from_pretrained(
    "./gpt4chan_model_float16", torch_dtype=torch.float16, low_cpu_mem_usage=True
)
model.cuda()
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

prompt = (
    "In a shocking finding, scientists discovered a herd of unicorns living in a remote, "
    "previously unexplored valley, in the Andes Mountains. Even more surprising to the "
    "researchers was the fact that the unicorns spoke perfect English."
)

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.cuda()

gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.8,
    top_p=0.9,
    max_length=100,
)
gen_text = tokenizer.batch_decode(gen_tokens)[0]


#float32 (cpu)

In [ ]:
!wget https://archive.org/download/gpt4chan_model/gpt4chan_model_archive.torrent
!wget https://github.com/Aspie96/gpt-4chan-model/main/float16/config.json



In [ ]:
from torrentp import TorrentDownloader
torrent_file = TorrentDownloader("gpt4chan_model_archive.torrent", '.')
await torrent_file.start_download()
!cp config.json ./gpt4chan_model/

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("./gpt4chan_model")
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

prompt = (
    "In a shocking finding, scientists discovered a herd of unicorns living in a remote, "
    "previously unexplored valley, in the Andes Mountains. Even more surprising to the "
    "researchers was the fact that the unicorns spoke perfect English."
)

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.8,
    top_p=0.9,
    max_length=100,
)
gen_text = tokenizer.batch_decode(gen_tokens)[0]